# TruLens for LLMs: Quickstart

In this quickstart you will create a simple LLM Chain and learn how to log it and get feedback on an LLM response.

Note: If you haven't already, make sure to set up your local .env file with your OpenAI and Huggingface Keys. Your .env file should be in the same directory that you run this notebook. If you need help, take a look at the [.env.example](https://github.com/truera/trulens_private/blob/e8b11c4689e644687d2eafe09d90d8d7774b581c/trulens_eval/trulens_eval/.env.example#L4)

## Import from LangChain and TruLens

In [ ]:
# imports from langchain to build app
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
# imports from trulens to log and get feedback on chain
from trulens_eval import tru
from trulens_eval import tru_chain
from trulens_eval.tru_feedback import Feedback, Huggingface, OpenAI
from trulens_eval.keys import *

## Create Simple LLM Application

This example uses a LangChain framework and OpenAI LLM

In [ ]:
full_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="Provide a helpful response with relevant background information for the following: {prompt}",
            input_variables=["prompt"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.9)

chain = LLMChain(llm=chat, prompt=chat_prompt_template)

# wrap with truchain
tc = tru_chain.TruChain(chain)

## Send your first request to your new app, asking how to adopt a dog in spanish

In [ ]:
prompt_input = 'como adopto un perro?'
gpt3_response, record = tc.call_with_record(prompt_input)

## Log data and feedback

In [ ]:
record_id = tru.add_data(
        chain_id='Chain1_ChatApplication', #name your chain
        prompt=prompt_input, # prompt input
        response=gpt3_response['text'], # LLM response
        record=record, # record is returned by the TruChain wrapper
        tags='dev' #add a tag
    )


# initialize Huggingface class for feedback function generation
hugs = Huggingface()

# Generate a language match feedback function using HuggingFace
f_lang_match = Feedback(hugs.language_match).on(text1="prompt", text2="response")

# Run feedack functions
feedback = tru.run_feedback_functions(
        chain=chain, # the unwrapped chain
        record=record, # record is returned by the TruChain wrapper
        feedback_functions=[f_lang_match] # a list of feedback functions to apply
    )

tru.add_feedback(record_id, feedback) # log the feedback by providing the record id

## Run the TruLens dashboard to explore the quality of your LLM chain

In [ ]:
tru.run_dashboard() # open a local streamlit app to explore